# Imports

In [2]:
# To install OpenCV: type "conda install opencv" on terminal 
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import glob

from skimage.feature import corner_harris, corner_peaks
import cv2

if not os.path.exists("Q1"):
    os.makedirs("Q1")

/Users/chongyeegan/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


# 1.1 - Viewing Images as Movies

In [9]:
def animate_tool(frames, interval, repeat, repeat_delay):
    fig = plt.figure()
    im = plt.imshow(frames[0], cmap=plt.get_cmap('gray'), vmin=0, vmax=255, animated=True)
    
    def updatefig(j):
        im.set_array(frames[j])
        return im,

    ani = animation.FuncAnimation(fig, updatefig, frames=range(len(frames)), blit=True,
                                  interval=interval, repeat=repeat, repeat_delay=repeat_delay)
    return ani

In [ ]:
if __name__ == '__main__':
    path = 'Images/Q1'
    imagelist = []
    
    for filename in glob.glob(os.path.join(path, '*.bmp')): 
        im = plt.imread(filename)
        fn = filename.split('/')[-1].split('.')[0]
        imagelist.append((fn, im))

    print imagelist
    frames = [i[1] for i in imagelist]
    print frames
    
    # kick off the animation
    ani = animate_tool(frames, 100, True, 500)

    mins = []

# 1.2 - Harris Corner Detector 

In [45]:
%matplotlib inline
# this is skimage -- it looks like corner peaks does the filtering of repeat corners
# however, i can't make it output 100 corners for everything. not sure if it's worth trying cv2
# i tried a little but got confused (see comments below)

if __name__ == '__main__':
    path = 'Images/Q1'
    imagelist = []

    for filename in glob.glob(os.path.join(path, '*.bmp')): 
        im = cv2.imread(filename)
        fn = filename.split('/')[-1].split('.')[0]
        imagelist.append((fn, im))
    
    #OpenCV Method    
    for filename, frame in imagelist:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        gray = np.float32(gray)
        #dst = cv2.cornerHarris(gray,2,3,0.04)
        #img[dst>0.01*dst.max()]=[0,0,255]
        #cv2.imshow('dst',img)
        corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)
        corners = np.int0(corners)

        for i in corners:
            x,y = i.ravel()
            cv2.circle(frame,(x,y),3,255,-1)

        plt.imshow(frame)
        fn = filename + '_corner.png'
        plt.savefig('Q1/' + fn, dpi=200)
        plt.close()

        
    ''''
    #Skimage Method
    for filename, frame in imagelist:    
        ch = corner_harris(frame, k=0.04)
        coords = corner_peaks(ch, 1, 0)
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))

        ax.imshow(frame, interpolation='nearest', cmap=plt.cm.gray)
        ax.plot(coords[:, 1], coords[:, 0], '+r', markersize=15)
        plt.axis('off')
        fn = filename + '_corner.png'
        plt.savefig('Q1/' + fn, dpi=200)
        plt.close()
    '''
      


100
